# joins and relationships

## 1. introduction

* joins allows to leverage relationships between entities while doing queries.
* not all joins are supported by every technology (we will see workarounds).

## 2. sample database (same will be used for lab!)

In [43]:
# let's load jupyter sql extension

%load_ext sql
%config SqlMagic.autocommit = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [44]:
# load database

%sql sqlite:///data/publications.db

'Connected: @data/publications.db'

getting tables in publications database:

In [46]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///data/publications.db
Done.
Returning data to local variable tables


In [47]:
tables.DataFrame()

name
0       authors
1     discounts
2      employee
3          jobs
4      pub_info
5    publishers
6      roysched
7         sales
8        stores
9   titleauthor
10       titles

## 3. types of joins

### inner join

In [21]:
%%sql inner <<

SELECT 
pub_name,
count(titles.title_id) as titles_published
FROM publishers
JOIN titles
ON publishers.pub_id = titles.pub_id
GROUP BY pub_name
ORDER BY titles_published DESC;

 * sqlite:///data/publications.db
Done.
Returning data to local variable inner


In [32]:
inner.DataFrame()

pub_name  titles_published
0      Binnet & Hardley                 7
1  Algodata Infosystems                 6
2        New Moon Books                 5

### left join (all elements in the first table of the join will prevail)

In [48]:
%%sql left <<

SELECT 
pub_name,
count(titles.title_id) as titles_published
FROM publishers
LEFT JOIN titles
ON publishers.pub_id = titles.pub_id
GROUP BY pub_name
ORDER BY titles_published DESC;

 * sqlite:///data/publications.db
Done.
Returning data to local variable left


In [49]:
left.DataFrame()

pub_name  titles_published
0       Binnet & Hardley                 7
1   Algodata Infosystems                 6
2         New Moon Books                 5
3         Scootney Books                 0
4      Ramona Publishers                 0
5     Lucerne Publishing                 0
6                  GGG&G                 0
7  Five Lakes Publishing                 0

### right join (not supported by sqlite, just change order of tables)

In [27]:
%%sql right <<

select 
titles.title, 
titles.type, 
titles.price, 
sum(sales.qty) as units_sold
from titles
left join sales
on titles.title_id = sales.title_id
group by titles.title, titles.type, titles.price;

 * sqlite:///data/publications.db
Done.
Returning data to local variable right


In [30]:
right.DataFrame()

title          type  price  \
0                            But Is It User Friendly?  popular_comp  22.95   
1   Computer Phobic AND Non-Phobic Individuals: Be...    psychology  21.59   
2   Cooking with Computers: Surreptitious Balance ...      business  11.95   
3                 Emotional Security: A New Algorithm    psychology   7.99   
4           Fifty Years in Buckingham Palace Kitchens     trad_cook  11.95   
5                                 Is Anger the Enemy?    psychology  10.95   
6                                   Life Without Fear    psychology   7.00   
7                                       Net Etiquette  popular_comp    NaN   
8   Onions, Leeks, and Garlic: Cooking Secrets of ...     trad_cook  20.95   
9       Prolonged Data Deprivation: Four Case Studies    psychology  19.99   
10                          Secrets of Silicon Valley  popular_comp  20.00   
11                  Silicon Valley Gastronomic Treats      mod_cook  19.99   
12                      Straight Talk About Computers      business  19.99   
13                                     Sushi, Anyone?     trad_cook  14.99   
14                The Busy Executive's Database Guide      business  19.99   
15                              The Gourmet Microwave      mod_cook   2.99   
16                 The Psychology of Computer Cooking     UNDECIDED    NaN   
17                    You Can Combat Computer Stress!      business   2.99   

    units_sold  
0         30.0  
1         20.0  
2         25.0  
3         25.0  
4         20.0  
5        108.0  
6         25.0  
7          NaN  
8         40.0  
9         15.0  
10        50.0  
11        10.0  
12        15.0  
13        20.0  
14        15.0  
15        40.0  
16         NaN  
17        35.0

### outer join (not supported by sqlite and mysql either)

In [34]:
%%sql outer <<

SELECT 
employee.fname, 
employee.hire_date, 
jobs.job_desc, 
jobs.job_id
FROM jobs
LEFT JOIN employee
on jobs.job_id = employee.job_id
UNION
SELECT 
employee.fname, 
employee.hire_date, 
jobs.job_desc, 
jobs.job_id
FROM employee
LEFT JOIN jobs
on jobs.job_id = employee.job_id;

 * sqlite:///data/publications.db
Done.
Returning data to local variable outer


In [36]:
outer.DataFrame()

fname            hire_date                      job_desc  job_id
0        None                 None  New Hire - Job not specified       1
1     Anabela  1993-01-27 00:00:00      Public Relations Manager       8
2         Ann  1991-07-16 00:00:00   Business Operations Manager       3
3     Annette  1990-02-21 00:00:00               Managing Editor       6
4        Aria  1991-10-26 00:00:00           Productions Manager      10
5      Carine  1992-07-07 00:00:00          Sales Representative      13
6      Carlos  1989-04-21 00:00:00                     Publisher       5
7      Daniel  1990-01-01 00:00:00            Operations Manager      11
8       Diego  1991-12-16 00:00:00               Managing Editor       6
9   Elizabeth  1990-07-24 00:00:00                      Designer      14
10  Francisco  1990-11-03 00:00:00      Chief Financial Officier       4
11       Gary  1988-08-09 00:00:00          Acquisitions Manager       9
12      Helen  1989-09-21 00:00:00                        Editor      12
13  Helvetius  1993-03-19 00:00:00             Marketing Manager       7
14     Howard  1988-11-19 00:00:00                        Editor      12
15     Janine  1991-05-26 00:00:00                     Publisher       5
16      Karin  1992-10-17 00:00:00                      Designer      14
17      Karla  1994-03-11 00:00:00          Acquisitions Manager       9
18   Laurence  1990-06-03 00:00:00                     Publisher       5
19     Lesley  1991-02-13 00:00:00             Marketing Manager       7
20     Manuel  1989-01-09 00:00:00      Public Relations Manager       8
21   Margaret  1988-09-29 00:00:00          Acquisitions Manager       9
22      Maria  1989-03-01 00:00:00                     Publisher       5
23      Maria  1992-03-27 00:00:00             Marketing Manager       7
24     Martin  1990-04-13 00:00:00           Productions Manager      10
25    Martine  1992-02-05 00:00:00          Acquisitions Manager       9
26       Mary  1993-06-29 00:00:00      Public Relations Manager       8
27      Matti  1994-05-01 00:00:00               Managing Editor       6
28     Miguel  1992-12-07 00:00:00            Operations Manager      11
29      Palle  1993-05-09 00:00:00             Marketing Manager       7
30      Paolo  1992-08-27 00:00:00          Sales Representative      13
31   Patricia  1989-08-01 00:00:00            Operations Manager      11
32       Paul  1993-08-19 00:00:00                     Publisher       5
33      Paula  1994-01-19 00:00:00      Public Relations Manager       8
34      Pedro  1990-12-24 00:00:00                      Designer      14
35      Peter  1992-05-17 00:00:00           Productions Manager      10
36     Philip  1989-11-11 00:00:00       Chief Executive Officer       2
37     Pirkko  1993-11-29 00:00:00           Productions Manager      10
38       Rita  1993-10-09 00:00:00                     Publisher       5
39     Roland  1991-09-05 00:00:00            Operations Manager      11
40       Sven  1991-04-05 00:00:00                     Publisher       5
41    Timothy  1988-06-19 00:00:00          Sales Representative      13
42   Victoria  1990-09-13 00:00:00               Managing Editor       6
43      Yoshi  1989-06-11 00:00:00                        Editor      12

## 4. combined queries (with)

In [39]:
%%sql with_example <<

WITH 
employees_custom AS 
(
SELECT *
FROM employee
),
jobs_custom AS
(
SELECT *
FROM jobs
JOIN employees_custom
ON employees_custom.job_id = jobs.job_id
)
SELECT * from jobs_custom;

 * sqlite:///data/publications.db
Done.
Returning data to local variable with_example


In [40]:
with_example.DataFrame()

job_id                     job_desc  min_lvl  max_lvl     emp_id  \
0       10          Productions Manager       75      165  A-C71970F   
1        6              Managing Editor      140      225  A-R89858F   
2        3  Business Operations Manager      175      225  AMD15433F   
3        8     Public Relations Manager      100      175  ARD36773F   
4        5                    Publisher      150      250  CFH28514M   
5       13         Sales Representative       25      100  CGS88322F   
6       11           Operations Manager       75      150  DBT39435M   
7        6              Managing Editor      140      225  DWR65030M   
8       14                     Designer       25      100  ENL44273F   
9        4     Chief Financial Officier      175      250  F-C16315M   
10       9         Acquisitions Manager       75      175  GHT50241M   
11      12                       Editor       25      100  H-B39728F   
12       7            Marketing Manager      120      200  HAN90777M   
13      12                       Editor       25      100  HAS54740M   
14       5                    Publisher      150      250  JYL26161F   
15      14                     Designer       25      100  KFJ64308F   
16       9         Acquisitions Manager       75      175  KJJ92907F   
17       7            Marketing Manager      120      200  L-B31947F   
18       5                    Publisher      150      250  LAL21447M   
19       7            Marketing Manager      120      200  M-L67958F   
20       8     Public Relations Manager      100      175  M-P91209M   
21       9         Acquisitions Manager       75      175  M-R38834F   
22      11           Operations Manager       75      150  MAP77183M   
23       9         Acquisitions Manager       75      175  MAS70474F   
24      10          Productions Manager       75      165  MFS52347M   
25       6              Managing Editor      140      225  MGK44605M   
26       5                    Publisher      150      250  MJP25939M   
27       8     Public Relations Manager      100      175  MMS49649F   
28      11           Operations Manager       75      150  PCM98509F   
29       7            Marketing Manager      120      200  PDI47470M   
30      10          Productions Manager       75      165  PHF38899M   
31      13         Sales Representative       25      100  PMA42628M   
32      10          Productions Manager       75      165  POK93028M   
33      14                     Designer       25      100  PSA89086M   
34       8     Public Relations Manager      100      175  PSP68661F   
35       2      Chief Executive Officer      200      250  PTC11962M   
36       5                    Publisher      150      250  PXH22250M   
37      11           Operations Manager       75      150  R-M53550M   
38       5                    Publisher      150      250  RBM23061F   
39       5                    Publisher      150      250  SKO22412M   
40      13         Sales Representative       25      100  TPO55093M   
41       6              Managing Editor      140      225  VPA30890F   
42      12                       Editor       25      100  Y-L77953M   

        fname minit      lname  job_id:1  job_lvl pub_id            hire_date  
0        Aria             Cruz        10       87   1389  1991-10-26 00:00:00  
1     Annette           Roulet         6      152   9999  1990-02-21 00:00:00  
2         Ann     M      Devon         3      200   9952  1991-07-16 00:00:00  
3     Anabela     R  Domingues         8      100   0877  1993-01-27 00:00:00  
4      Carlos     F   Hernadez         5      211   9999  1989-04-21 00:00:00  
5      Carine     G    Schmitt        13       64   1389  1992-07-07 00:00:00  
6      Daniel     B     Tonini        11       75   0877  1990-01-01 00:00:00  
7       Diego     W       Roel         6      192   1389  1991-12-16 00:00:00  
8   Elizabeth     N    Lincoln        14       35   0877  1990-07-24 00:00:00  
9   Francisco            Chang     

## 5. lab time! let's start together

In [41]:
%%sql challenge_1 <<

select authors.au_id, 
authors.au_lname, 
authors.au_fname, 
titles.title
from titles
JOIN titleauthor
on titles.title_id = titleauthor.title_id
JOIN authors
on authors.au_id = titleauthor.au_id;

 * sqlite:///data/publications.db
Done.
Returning data to local variable challenge_1


In [42]:
challenge_1.DataFrame()

au_id        au_lname  au_fname  \
0   172-32-1176           White   Johnson   
1   213-46-8915           Green  Marjorie   
2   213-46-8915           Green  Marjorie   
3   238-95-7766          Carson    Cheryl   
4   267-41-2394         O'Leary   Michael   
5   267-41-2394         O'Leary   Michael   
6   274-80-9391        Straight      Dean   
7   409-56-7008          Bennet   Abraham   
8   427-17-2319            Dull       Ann   
9   472-27-2349      Gringlesby      Burt   
10  486-29-1786        Locksley  Charlene   
11  486-29-1786        Locksley  Charlene   
12  648-92-1872  Blotchet-Halls  Reginald   
13  672-71-3249        Yokomoto     Akiko   
14  712-45-1867    del Castillo     Innes   
15  722-51-5454        DeFrance    Michel   
16  724-80-9391      MacFeather   Stearns   
17  724-80-9391      MacFeather   Stearns   
18  756-30-7391          Karsen     Livia   
19  807-91-6654        Panteley    Sylvia   
20  846-92-7186          Hunter    Sheryl   
21  899-46-2035          Ringer      Anne   
22  899-46-2035          Ringer      Anne   
23  998-72-3567          Ringer    Albert   
24  998-72-3567          Ringer    Albert   

                                                title  
0       Prolonged Data Deprivation: Four Case Studies  
1                 The Busy Executive's Database Guide  
2                     You Can Combat Computer Stress!  
3                            But Is It User Friendly?  
4   Cooking with Computers: Surreptitious Balance ...  
5                                      Sushi, Anyone?  
6                       Straight Talk About Computers  
7                 The Busy Executive's Database Guide  
8                           Secrets of Silicon Valley  
9                                      Sushi, Anyone?  
10                                      Net Etiquette  
11                Emotional Security: A New Algorithm  
12          Fifty Years in Buckingham Palace Kitchens  
13                                     Sushi, Anyone?  
14                  Silicon Valley Gastronomic Treats  
15                              The Gourmet Microwave  
16  Cooking with Computers: Surreptitious Balance ...  
17  Computer Phobic AND Non-Phobic Individuals: Be...  
18  Computer Phobic AND Non-Phobic Individuals: Be...  
19  Onions, Leeks, and Garlic: Cooking Secrets of ...  
20                          Secrets of Silicon Valley  
21                              The Gourmet Microwave  
22                                Is Anger the Enemy?  
23                                Is Anger the Enemy?  
24                                  Life Without Fear